In [ ]:
#Keep The Data
import cv2
import os

folder_name = "DataSet_Me/person1"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Load the face detector (Haarcascade)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Open the video capture
cap = cv2.VideoCapture(0)

# Counter for the number of images captured
img_count = 0

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    if not ret:
        print("Cannot read frame from the camera")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Draw rectangles around detected faces
    for x, y, w, h in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Capture and save the face image in the DataSet_Me folder
        face_img = frame[y : y + h, x : x + w]
        img_count += 1
        img_path = os.path.join(folder_name, f"face_{img_count}.jpg")
        cv2.imwrite(img_path, face_img)
        print(f"Saved image {img_path}")

    # Display the current frame
    cv2.imshow("Keeping Data......", frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the camera and close the windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
#Check file
import os

data_dir = "DataSet_Me"
if not os.path.exists(data_dir):
    print(f"Folder {data_dir} not found")
else:
    subfolders = os.listdir(data_dir)
    print(f"Subfolders in {data_dir}: {subfolders}")

    for subfolder in subfolders:
        subfolder_path = os.path.join(data_dir, subfolder)
        if os.path.isdir(subfolder_path):
            files = os.listdir(subfolder_path)
            print(f"Files in {subfolder_path}: {files}")

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size
img_height = 128
img_width = 128

# Define batch size
batch_size = 32

# Path to the data folder
train_dir = "DataSet_Me"  # Folder containing face data divided into person1 and unknown

# Create ImageDataGenerator with data augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to 0-1
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

# Create data generator for training data
data_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),  # Define the desired image size
    batch_size=batch_size,
    class_mode="binary",  # Binary classification (person1 and unknown)
)

# Build CNN model
model = models.Sequential(
    [
        layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)
        ),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(1, activation="sigmoid"),  # Sigmoid for binary classification
    ]
)

# Compile the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",  # Binary cross-entropy for binary classification
    metrics=["accuracy"],
)

# Train the model
history = model.fit(data_gen, epochs=20, steps_per_epoch=data_gen.samples // batch_size)

# Save the trained model
model.save("face_recognition_model.h5")

print("Model trained and saved successfully!")

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("face_recognition_model.h5")

# Load the face detector (Haarcascade)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Open video capture
cap = cv2.VideoCapture(0)

# Image parameters (using 128x128 to match the model)
img_height, img_width = 128, 128

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    if not ret:
        print("Cannot read frame from the camera")
        break

    # Convert the frame to grayscale to speed up face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for x, y, w, h in faces:
        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Crop the face from the frame for prediction
        face_img = frame[y : y + h, x : x + w]
        face_img = cv2.resize(
            face_img, (img_height, img_width)
        )  # Resize the image to fit the model
        face_img = np.expand_dims(face_img, axis=0)  # Add batch dimension
        face_img = face_img / 255.0  # Normalize

        # Use the model to predict if the face is known or unknown
        prediction = model.predict(face_img)

        # Display the result of the prediction
        if prediction[0] > 0.1:
            label = "Known Face"
        else:
            label = "Unknown Face"

        # Put the label on the face
        cv2.putText(
            frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2
        )

    # Show the current frame
    cv2.imshow("Face Recognition", frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the camera and close the windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("face_recognition_model.h5")

# Load the face detector (Haarcascade)
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Open video capture
cap = cv2.VideoCapture(0)

# Image parameters
img_height, img_width = 128, 128

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    if not ret:
        print("Cannot read frame from the camera")
        break

    # Convert the frame to grayscale to speed up face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    known_face_detected = False

    for x, y, w, h in faces:
        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Crop the face from the frame for prediction
        face_img = frame[y : y + h, x : x + w]
        face_img = cv2.resize(
            face_img, (img_height, img_width)
        )  # Resize the image to fit the model
        face_img = np.expand_dims(face_img, axis=0)  # Add batch dimension
        face_img = face_img / 255.0  # Normalize

        # Use the model to predict if the face is known or unknown
        prediction = model.predict(face_img)

        # Display the result of the prediction
        if prediction[0] > 0.5:  # Adjust threshold as needed
            label = "Known Face"
            known_face_detected = True
        else:
            label = "Unknown Face"

        # Put the label on the face
        cv2.putText(
            frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2
        )

    # Draw a circle at the top-right corner of the window
    if known_face_detected:
        cv2.circle(
            frame, (frame.shape[1] - 30, 30), 20, (0, 255, 0), -1
        )  # Green circle
    else:
        cv2.circle(frame, (frame.shape[1] - 30, 30), 20, (0, 0, 255), -1)  # Red circle

    # Show the current frame
    cv2.imshow("Face Recognition", frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the camera and close the windows
cap.release()
cv2.destroyAllWindows()